In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
import math
from scipy import stats
plt.rc("figure", figsize=(16,8))
plt.rc("font", size=14)

In [2]:
train_df=pd.read_csv('/Users/Ben/Desktop/optimal_decision_trees/data/modelling.csv')
holdout_df=pd.read_csv('/Users/Ben/Desktop/optimal_decision_trees/data/holdout.csv')
kaggle_df=pd.read_csv('/Users/Ben/Desktop/optimal_decision_trees/data/test.csv')

In [3]:
print(train_df.shape)
print(holdout_df.shape)
print(kaggle_df.shape)

(160001, 116)
(28317, 116)
(125546, 131)


# Data partition

In [4]:
def training_validation_subset(df):
    ''' function to create training and validation subsets
        chosen this methodology as a method to replicate in the future '''

    training_df = df.sample(frac=0.7)
    print('Modelling dataset rows:\t', training_df.shape[0])

    validation_df = pd.concat([df, training_df]).drop_duplicates(keep=False)
    print('Validation dataset rows:\t', validation_df.shape[0])

    return training_df, validation_df

modelling_df, validation_df=training_validation_subset(train_df)

Modelling dataset rows:	 112001
Validation dataset rows:	 48000


# Feature engineering

In [5]:
def response_outlier_capping(df, variable, multiplier):

    ''' windsorise the response variable '''

    q1 = np.percentile(df[variable],25)
    q3 = np.percentile(df[variable],75)
    iqr = q3 - q1
    lower = q1 - (iqr * multiplier)
    upper = q3 + (iqr * multiplier)

    df[variable] = np.where(df[variable]<=lower, lower, df[variable])
    df[variable] = np.where(df[variable]>=upper, upper, df[variable])

    return df

def log_response(df, response):

    ''' take the natural log of the response variable '''

    print('Skewness of untransformed response:\t' + str(df[response].skew()))

    # transform response column to ensure +ve
    minimum_val = math.ceil(min(abs(np.log(df[response]))))
    original_data = np.log(df[response]) + minimum_val
    df[response] = np.log(df[response])
    print('Skewness of transformed response:\t' + str(df[response].skew()))

    return df

modelling_df=response_outlier_capping(modelling_df, 'loss', 2.2)

In [6]:
def return_categoric_columns(df):
    
    ''' function to return categoric columns '''

    all_columns = list(df.columns)
    numeric_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'uint8']
    numeric_columns = df.select_dtypes(include=numeric_types).columns.to_list()
    categoric_columns = list(set(all_columns) - set(numeric_columns))
    return categoric_columns

In [7]:
def frequency_bin_cols(df, threshold):
    
    ''' function to group up values with low frequency based on a specified threshold'''

    # empty lists
    fe_cols=[]
    fe_transform=[]

    for i in categoric_columns:

        if len(df[i].unique())>=8:

            fe_cols.append(i)
            fe_transform.append(df[i].map(df.groupby(i)[i].size().sort_values(ascending=False).div(len(df)).cumsum().le(threshold)))

    for u, v in zip(fe_cols, fe_transform):
        
        df[u] = np.where(v, df[u], 'Other')

In [8]:
categoric_columns=return_categoric_columns(modelling_df)
frequency_bin_cols(modelling_df, 0.8)

In [9]:
def frequency_transformer(df1, df2, col):

    ''' function to transform datasets to match '''
    
    list1=list(sorted(df1[col].unique()))
    list2=list(sorted(df2[col].unique()))

    intersection=list(set(list1).intersection(list2))

    df2[col]=np.where(df2[col].isin(intersection),df2[col],'Other')

In [10]:
for i in categoric_columns:
    frequency_transformer(modelling_df, validation_df, i)

# Modelling

This part can be improved by automating the glm build, e.g. https://planspace.org/20150423-forward_selection_with_statsmodels/

In [11]:
def glm_iteration(df, starting_formula, iterable_cols):
    
    ''' function to build the an intertion of a glm '''

    column=[]
    formulas=[]
    pvalues=[]

    for i in iterable_cols:

        formula=starting_formula

        if i in (return_categoric_columns(df)):

            column.append(i)
            formula=formula+'C('+i+')'
            formulas.append(formula)
            glm_gamma = smf.glm(formula=formula, data=df, family=sm.families.Gamma(sm.families.links.log()))
            glm_results = glm_gamma.fit()
            pvalues.append(list(glm_results.pvalues)[-1])

        else: 

            column.append(i)
            formula=formula+i
            formulas.append(formula)
            glm_gamma = smf.glm(formula=formula, data=df, family=sm.families.Gamma(sm.families.links.log()))
            glm_results = glm_gamma.fit()
            pvalues.append(list(glm_results.pvalues)[-1])

    min_pvalue_index=pvalues.index(min(pvalues))
    fitted_factor=column[min_pvalue_index]
    fitted_formula=formulas[min_pvalue_index]
    fitted_pvalue=pvalues[min_pvalue_index]
            
    return fitted_factor,fitted_formula,fitted_pvalue

In [12]:
def automate_glm(df, starting_formula, iterable_cols):
    
    ''' function to build many glm models and continue adding variables based on p values '''
    
    fitted_factor,fitted_formula,fitted_pvalue=glm_iteration(df, starting_formula, iterable_cols)
    print(fitted_factor,fitted_formula,fitted_pvalue)

    for i in range(0,len(iterable_cols)-1):
        
        while fitted_pvalue<=0.05:
        
            iteration_formula=fitted_formula+'+'
            iterable_cols.remove(fitted_factor)
            fitted_factor,fitted_formula,fitted_pvalue=glm_iteration(df, iteration_formula, iterable_cols)
            print(fitted_factor,fitted_formula,fitted_pvalue)
    
    best_formula=fitted_formula.replace('+'+fitted_factor, "")
    return best_formula

In [13]:
removal_cols=['id', 'loss']
X=modelling_df.drop(removal_cols, axis=1)
y=np.log(modelling_df['loss'])
iterable_cols = list(X[X.columns[0:18]])
starting_formula='y~'
modelling_df

best_formula = automate_glm(modelling_df, starting_formula, iterable_cols)

cat1 y~C(cat1) 0.0
cat10 y~C(cat1)+C(cat10) 0.0
cont2 y~C(cat1)+C(cat10)+cont2 1.509188610284137e-300
cat104 y~C(cat1)+C(cat10)+cont2+C(cat104) 0.0
cat100 y~C(cat1)+C(cat10)+cont2+C(cat104)+C(cat100) 0.0
cont7 y~C(cat1)+C(cat10)+cont2+C(cat104)+C(cat100)+cont7 5.9766667931805126e-238
cont3 y~C(cat1)+C(cat10)+cont2+C(cat104)+C(cat100)+cont7+cont3 1.1028072507200276e-35
cont14 y~C(cat1)+C(cat10)+cont2+C(cat104)+C(cat100)+cont7+cont3+cont14 1.0831188147565186e-32
cont4 y~C(cat1)+C(cat10)+cont2+C(cat104)+C(cat100)+cont7+cont3+cont14+cont4 4.980649168784691e-07
cont8 y~C(cat1)+C(cat10)+cont2+C(cat104)+C(cat100)+cont7+cont3+cont14+cont4+cont8 2.3047210160066206e-13
cont12 y~C(cat1)+C(cat10)+cont2+C(cat104)+C(cat100)+cont7+cont3+cont14+cont4+cont8+cont12 3.067013911637932e-08
cont10 y~C(cat1)+C(cat10)+cont2+C(cat104)+C(cat100)+cont7+cont3+cont14+cont4+cont8+cont12+cont10 1.740914905701353e-14
cont9 y~C(cat1)+C(cat10)+cont2+C(cat104)+C(cat100)+cont7+cont3+cont14+cont4+cont8+cont12+cont10+cont9

In [14]:
glm_gamma = smf.glm(formula=best_formula, data=modelling_df, family=sm.families.Gamma(sm.families.links.log()))
glm_results = glm_gamma.fit()

In [15]:
X_train=modelling_df.drop(removal_cols, axis=1)
y_pred=np.exp(glm_results.predict(X_train))
y_valid=modelling_df['loss']
result=mean_absolute_error(y_pred, y_valid)
print('result: ', result)

result:  1495.2783190564853


In [16]:
X_test=validation_df.drop(removal_cols, axis=1)
y_pred=np.exp(glm_results.predict(X_test))
y_valid=validation_df['loss']
result=mean_absolute_error(y_pred, y_valid)
print('result: ', result)

result:  1626.4597339043573


# Score holdout

In [17]:
holdout_cats=return_categoric_columns(holdout_df)

for i in holdout_cats:

    frequency_transformer(modelling_df, holdout_df, i)

In [18]:
y_pred=np.exp(glm_results.predict(holdout_df))
idx=holdout_df['id']
d = {'id':idx,'glm_pred':y_pred}
out_df=pd.DataFrame(d)
out_df.to_csv('/Users/Ben/Desktop/optimal_decision_trees/outputs/holdout_glm_predictions.csv', index=False)

In [19]:
kaggle_cats=return_categoric_columns(kaggle_df)

for i in holdout_cats:

    frequency_transformer(modelling_df, kaggle_df, i)

In [20]:
y_pred=np.exp(glm_results.predict(kaggle_df))
idx=kaggle_df['id']
d = {'id':idx,'loss':y_pred}
out_df=pd.DataFrame(d)
out_df.to_csv('/Users/Ben/Desktop/optimal_decision_trees/outputs/kaggle_glm_predictions.csv', index=False)

In [21]:
params=glm_results.params.to_frame().reset_index()
params.columns=['level', 'coef']
params.to_csv('/Users/Ben/Desktop/optimal_decision_trees/outputs/glm_params.csv', index=False)

In [22]:
glm_results.summary().as_csv()

results_summary = glm_results.summary()

# Note that tables is a list. The table at index 1 is the "core" table. Additionally, read_html puts dfs in a list, so we want index 0
results_as_html = results_summary.tables[1].as_html()

with open("/Users/Ben/Desktop/optimal_decision_trees/outputs/glm_summary.html", "w") as file:
    file.write(results_as_html)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:735: RuntimeWarning: invalid value encountered in log
  ll_obs -= special.gammaln(weight_scale) + np.log(endog)
